In [1]:
from aiida import load_profile
from aiida.orm import Int
from aiida_workgraph import task, WorkGraph

In [2]:
from pickle import loads

In [3]:
@task.calcfunction()
def add_x_and_y(x, y):
    z = x + y
    return z

In [4]:
@task.calcfunction()
def add_x_and_y_and_z(x, y, z):
    w = x + y + z
    return w

In [5]:
load_profile()

Profile<uuid='7bb8761123324468bb98821cbb757251' name='presto'>

In [6]:
wg = WorkGraph("my_workflow")
wg.add_task(add_x_and_y, name="add_x_and_y", x=1, y=2)
wg.add_task(add_x_and_y_and_z, name="add_x_and_y_and_z", z=wg.tasks["add_x_and_y"].outputs["result"], x=1, y=2)

Widget dependency not found. To visualize the workgraph, please install the widget dependency. Use 'pip install aiida-workgraph[widget]' if installing from PyPI. For local source installations, use 'pip install .[widget]' and then build the JavaScript library. Refer to the documentation for more details.


DecoratedNode(name='add_x_and_y_and_z', properties=[], inputs=['metadata', 'metadata.store_provenance', 'metadata.description', 'metadata.label', 'metadata.call_link_label', 'metadata.disable_cache', 'x', 'y', 'z', '_wait'], outputs=['result', '_outputs', '_wait'])

In [7]:
work_graph_dict = wg.to_dict()

In [8]:
edges_label_lst = []
for link_dict in work_graph_dict["links"]:
    if link_dict['from_socket'] == "result":
        edges_label_lst.append({'target': link_dict['to_node'], 'targetHandle': link_dict['to_socket'], 'source': link_dict['from_node'], 'sourceHandle': None})
    else:
        edges_label_lst.append({'target': link_dict['to_node'], 'targetHandle': link_dict['to_socket'], 'source': link_dict['from_node'], 'sourceHandle': link_dict['from_socket']})

edges_label_lst

[{'target': 'add_x_and_y_and_z',
  'targetHandle': 'z',
  'source': 'add_x_and_y',
  'sourceHandle': None}]

In [9]:
kwargs_dict, function_dict = {}, {}
for task_name, task_dict in work_graph_dict["tasks"].items():
    input_variables = [
        input_parameter 
        for input_parameter in task_dict['inputs'].keys() 
        if not input_parameter.startswith("metadata") and not input_parameter.startswith("_wait")
    ]
    input_kwargs = {input_parameter: task_dict['inputs'][input_parameter]['property']["value"].value if isinstance(task_dict['inputs'][input_parameter]['property']["value"], dict) else task_dict['inputs'][input_parameter]['property']["value"] for input_parameter in input_variables}
    kwargs_dict[task_name] = input_kwargs
    function_dict[task_name] = loads(task_dict['executor']['executor'])
kwargs_dict, function_dict

({'add_x_and_y': {'x': 1, 'y': 2},
  'add_x_and_y_and_z': {'x': 1, 'y': 2, 'z': None}},
 {'add_x_and_y': <function __main__.add_x_and_y(x, y)>,
  'add_x_and_y_and_z': <function __main__.add_x_and_y_and_z(x, y, z)>})

In [10]:
function_dict['add_x_and_y'](1, 2)

<Int: uuid: f9f34784-b574-4352-b3b4-4da535f87b92 (pk: 36) value: 3>

In [11]:
work_graph_dict 

{'version': 'node_graph@0.0.1',
 'uuid': '4c033048-ad58-11ef-b01a-9ec6524ec879',
 'name': 'my_workflow',
 'state': 'CREATED',
 'action': 'NONE',
 'error': '',
 'metadata': {'type': 'NORMAL',
  'platform': 'node_graph',
  'group_properties': [],
  'group_inputs': [],
  'group_outputs': []},
 'links': [{'from_socket': 'result',
   'from_node': 'add_x_and_y',
   'to_socket': 'z',
   'to_node': 'add_x_and_y_and_z',
   'state': False}],
 'ctrl_links': [],
 'description': '',
 'log': '',
 'context': {'_sequence': []},
 'restart_process': None,
 'max_iteration': 1000000,
 'execution_count': 0,
 'workgraph_type': 'NORMAL',
 'conditions': [],
 'max_number_jobs': 1000000,
 'error_handlers': {},
 'tasks': {'add_x_and_y': {'version': 'node_graph@0.0.1',
   'identifier': 'ADD_X_AND_Y',
   'uuid': '4c0336e2-ad58-11ef-b01a-9ec6524ec879',
   'name': 'add_x_and_y',
   'list_index': 1,
   'state': 'PLANNED',
   'action': '',
   'error': '',
   'metadata': {'node_type': 'CALCFUNCTION',
    'catalog': 'Ot

In [12]:
wg.submit(
    inputs={"add_x_and_y": {"x": Int(1), "y": Int(2)}, "add_x_and_y_and_z": {"x": Int(1), "y": Int(2)}}, 
    wait=True
)

WorkGraph process created, PK: 41


ConfigurationError: profile `presto` does not provide a communicator because it does not define a broker